<a href="https://colab.research.google.com/github/PedroDS4/Tabela_Routh_Hurwitz_Estabilidade_SLIT/blob/main/Implementa%C3%A7%C3%A3o_modelagem_estabilidade.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##**Atividade Modelagem**
**Implementação computacional do cálculo da tabela de Routh-Hurwitz**
Seja um sistema LIT representado pela seguinte função de transferência

$$
G(s) = \frac{N(s)}{D(s)} = \frac{N(s)}{\sum_{j = 0}^{M} a_j \cdot s^j}
$$

dizemos que o sistema é estável se todas as raízes de $D(s)$ estão no semiplano complexo esquerdo, ou seja, todas tem parte real menos que zero.

Para isso existem métodos para verificar a estabilidade desses sistemas, um deles é a tabela de Routh Hurwitz, que nos permite por meio de alguns cálculos verificar se um sistema é BIBO-estável ou não.

A tabela para o polinômio genérico $D(s)$ é dada por

\begin{array}{c|cccc}
s^n & a_n & a_{n-2} & a_{n-4} & \cdots \\
s^{n-1} & a_{n-1} & a_{n-3} & a_{n-5} & \cdots \\
s^{n-2} & c_1 & c_2 & c_3 & \cdots \\
\vdots & \vdots & \vdots & \vdots & \ddots \\
s^1 & \cdot & \cdot & \cdot & \\
s^0 & \cdot & \cdot & \cdot &
\end{array}


onde os termos abaixo das duas primeiras linhas seguem a lógica
$$
c_1 =
\begin{vmatrix}
a_n & a_{n-2} \\
a_{n-1} & a_{n-3}
\end{vmatrix}
$$

Por fim verificamos se há mudança de sinal na primeira coluna da tabela montada, em casa afirmativo dizemos que o sistema não é BIBO-estável e o número de raízes no semi-plano direito é igual ao número de mudanças de sinal.
Em caso negativo, dizemos que o sistema é BIBO-estável.







In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import numpy.linalg as la


def RH(coeficientes):
  n = len(coeficientes)
  grau = n - 1
  m = (n+1)//2
  right_semi_plane_roots = 0
  is_stable = True


  if grau == 1:
    raiz = -coeficientes[1]/coeficientes[0]

    if raiz>0:
      right_semi_plane_roots = 1
      left_semi_plane_roots = 1-right_semi_plane_roots
      is_stable = False
      return np.array([  [coeficientes[0],0],[coeficientes[1],0]  ]), is_stable, right_semi_plane_roots, left_semi_plane_roots

  tabela = np.zeros((n, m))

  first_line = coeficientes[0::2]
  second_line = coeficientes[1::2]


  tabela[0] = first_line

  if grau % 2 == 0:
    tabela[1] = np.concatenate((second_line,np.zeros(1)))
  else:
    tabela[1] = second_line

  for i in range(2, n):
        for j in range(m - 1):  # Ajustado para evitar IndexError
            # Cria as fatias de tamanho 2 que representam as colunas acima
            slice2 = tabela[i - 1, j:j + 2]
            slice1 = tabela[i - 2, j:j + 2]

            # Verifica se as fatias têm tamanho 2 antes de calcular o determinante
            if len(slice1) == 2 and len(slice2) == 2:
                # Constrói a matriz 2x2 e calcula o determinante
                matrix = np.array([slice1, slice2])
                c_ij = -la.det(matrix) / tabela[i - 1, j]
                tabela[i, j] = c_ij

            else:
                # Lida com casos onde a fatia tem menos de 2 elementos
                # (por exemplo, no final da linha)
                tabela[i, j] = 0  # ou outro valor apropriado



  for i in range(n):
    if np.sign(tabela[i,0]) != np.sign(tabela[0,0]):
      right_semi_plane_roots += 1
      is_stable = False
    if tabela[i,0] == 0:
      is_stable = False


  left_semi_plane_roots = grau - right_semi_plane_roots


  return tabela, is_stable, right_semi_plane_roots, left_semi_plane_roots




def exibir_tabela(tabela):
  """
  Exibe a tabela de Routh-Hurwitz formatada com potências de 's'.

  Args:
    tabela: A tabela de Routh-Hurwitz como um array NumPy.
  """
  n_linhas = tabela.shape[0]
  n_colunas = tabela.shape[1]
  grau = n_linhas - 1

  # Imprime o cabeçalho com as potências de 's'
  print(" " * 5, end="")  # Espaçamento inicial
  for j in range(n_colunas):
    print(f"c{j+1:2d}", end="   ")  # Cabeçalho das colunas
  print()

  for i in range(n_linhas):
    # Calcula a potência de 's' para a linha atual
    potencia_s = grau - i

    # Imprime a potência de 's'
    print(f"s^{potencia_s:2d}", end="  ")

    # Imprime os valores da linha
    for j in range(n_colunas):
      print(f"{tabela[i, j]}", end=" ")
    print()  # Nova linha


In [ ]:
#Exemplo com polinômio de segundo grau
#s^2 + 2s + 1

coef = np.array([4,1,2,1])
table, is_stable, right_semi_plane_roots, left_semi_plane_roots = RH(coef)
print(table)
print(is_stable)
print(right_semi_plane_roots)
print(left_semi_plane_roots)

exibir_tabela(table)

[[ 4.  2.]
 [ 1.  1.]
 [-2.  0.]
 [ 1.  0.]]
False
1
2
     c 1   c 2   
s^ 3  4.0 2.0 
s^ 2  1.0 1.0 
s^ 1  -2.0 0.0 
s^ 0  1.0 0.0 


In [ ]:
#Exemplo polinômio de grau 1
#s^3+s^2+2s+1
coef = np.array([1,1,2,1])
table, is_stable, right_semi_plane_roots, left_semi_plane_roots = RH(coef)
print(table)
print(is_stable)
print(right_semi_plane_roots)
print(left_semi_plane_roots)

exibir_tabela(table)

[[1. 2.]
 [1. 1.]
 [1. 0.]
 [1. 0.]]
True
0
3
     c 1   c 2   
s^ 3  1.0 2.0 
s^ 2  1.0 1.0 
s^ 1  1.0 0.0 
s^ 0  1.0 0.0 
